In [ ]:
from goatools import obo_parser
import gzip

In [ ]:
go_obo = '../DownloadedResources/go-basic.obo'
go = obo_parser.GODag(go_obo)

In [ ]:
len(go)

In [ ]:
go_id = 'GO:0048527'
go_term = go[go_id]
print(go_term.name, go_term.namespace)
go_term

In [ ]:
g = go_term
while True:
    print(g.name)
    if len(g.parents) == 0:
        break
    g = list(g.parents)[0]
    
    

In [ ]:
# go_terms = ['GO:0050776', 'GO:0002684']
go_terms = ['GO:0022904', 'GO:0022900', 'GO:0044271', 'GO:1901566', 'GO:0015934', 'GO:0030964', 
            'GO:0005747', 'GO:0045271', 'GO:0022625', 'GO:0015935' ]
recs = [go[t] for t in go_terms]
lineage_png = '../Cache/' + 'terms' + '-lineage.png'

In [ ]:
go.draw_lineage(recs, lineage_img=lineage_png,draw_children=False, dpi=300)

In [ ]:
from IPython.display import Image
Image(lineage_png)

In [ ]:
eg_term = '0042773'
eg_png_fname = '../Plots/GO_%s_lineage.png' % eg_term
eg_rec = go['GO:' + eg_term]
go.draw_lineage([eg_rec], lineage_img=eg_png_fname, draw_children=True, dpi=300)
Image(eg_png_fname)

In [ ]:
import Bio.UniProt.GOA as GOA

In [ ]:
with gzip.open('../DownloadedResources/goa_human.gaf.gz', 'rt') as gaf:
    funcs = {}
    for entry in GOA.gafiterator(gaf):
        uniprot_id = entry.pop('DB_Object_ID')
        funcs[uniprot_id] = entry

In [ ]:
list(funcs.items())[0]

## Enrichment analysis

In [ ]:
from goatools.go_enrichment import GOEnrichmentStudy

In [ ]:
pop = funcs.keys()

In [ ]:
assoc = {}
for x in funcs:
    if x not in assoc:
        assoc[x] = set()
    assoc[x].add(str(funcs[x]['GO_ID']))

In [ ]:
max([len(assoc[x]) for x in assoc.keys()])

In [ ]:
keyword = 'growth'
growth_dict = {x: funcs[x]
for x in funcs 
    if keyword in funcs[x]['DB_Object_Name']}

In [ ]:
growth_dict.keys()

In [ ]:
study = growth_dict.keys()

In [ ]:
methods = ["fdr"]

In [ ]:
g = GOEnrichmentStudy(pop, assoc, go,
                         propagate_counts=True,
                         alpha=0.05,
                         methods=methods)
g_res = g.run_study(study)

## Human

In [ ]:
with gzip.open('../DownloadedResources/goa_human.gaf.gz', 'rt') as gaf:
    human_funcs = {}
    for entry in GOA.gafiterator(gaf):
        if go[entry['GO_ID']].namespace == 'biological_process':
            symbol = entry.pop('DB_Object_Symbol')
            human_funcs[symbol] = entry

In [ ]:
pop = human_funcs.keys()

In [ ]:
[go[human_funcs[k]['GO_ID']].children for k in list(pop)[:3]]

In [ ]:
len(pop)

In [ ]:
for k in list(human_funcs)[:10]:
    print(go[human_funcs[k]['GO_ID']].namespace)

In [ ]:
assoc = {}
for x in human_funcs:
    if x not in assoc:
        assoc[x] = set()
    assoc[x].add(str(human_funcs[x]['GO_ID']))

In [ ]:
for gid in assoc['CXCL9']:
    print(go[gid])

In [ ]:
study0 = ['IGLL5', 'CXCL9', 'JCHAIN', 'UBD', 'ADAMDEC1', 'CXCL10', 'CXCL13', 'CXCL11', 'MZB1', 'HP', 'CD2', 'SLAMF7', 'GBP5', 'CD38', 'OR2I1P', 'PLA2G2D', 'CCL19', 'HOXD1', 'CSAG3', 'HLA-DQA1']
study1 = ['SFRP2', 'POSTN', 'EPYC', 'COL11A1', 'LRRC15', 'ADH1B', 'COL10A1', 'STAR', 'FAP', 'COMP', 'GREM1', 'FABP4', 'LUM', 'INHBA', 'TENM3', 'MMP11', 'NR5A1', 'ITGBL1', 'CXCL14', 'NTM']

In [ ]:
g = GOEnrichmentStudy(pop, assoc, go,
                         propagate_counts=True,
                         alpha=0.05,
                         methods=["fdr"])
g_res = g.run_study(study0)

In [ ]:
import sys
with open('../Cache/goa_results.tsv', 'w') as f:
    g.prt_tsv(f, g_res)

In [ ]:
## Synthetic

In [ ]:
pop = [str(chr(c)) for c in range(ord('a'), ord('z'))]

In [ ]:
assoc1 = {k:set(['GO:0090718']) for k in pop[:10]}
assoc2 = {k:set(['GO:0002459']) for k in pop[10:]}
assoc = {**assoc1, **assoc2}

In [ ]:
assoc

In [ ]:
assoc['a']

In [ ]:
study=['a','b','c','d','e', 'f', 'g']

In [ ]:
g = GOEnrichmentStudy(pop, assoc, go,
                         propagate_counts=True,
                         alpha=0.05,
                         methods=["fdr"])
g_res = g.run_study(study)

In [ ]:
g_res